In [47]:
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
from scipy.stats import skew, boxcox
from tensorflow.keras.models import load_model
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt 
from joblib import dump, load
import time
import itertools
import warnings
warnings.filterwarnings("ignore")

In [49]:
test_data=pd.read_csv("train.csv")

In [61]:
type(test_data)

pandas.core.frame.DataFrame

In [48]:
def final_fun_1(X):
    "The function which transforms all the raw input into predictions"
    X=X.drop(['loss'],axis=1)
    
    start = time.time()
    train_data=X.copy()
    
    top_cat_feats = "cat80,cat79,cat87,cat57,cat101,cat12,cat81,cat7,cat89,cat10,cat1,cat72,cat2,cat94,cat103,cat111,cat114,cat11,cat53,cat106,cat9,cat13,cat38,cat100,cat105,cat44,cat108,cat75,cat109,cat90,cat116,cat6,cat5,cat25".split(',')
    
# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--   
    def encode(string):
        '''Using unicode encoding to encode the categorical variables , this encoding uses relative position of the alphabet to encode the categorical variables'''
        r = 0
        length = len(str(string))
        for i in range(length):
            # unicode of the alphabet - unicode of first letter 
            # +1 to give maintain non zero postion
            # *26 for equating all the alphabets to a level as 26 is total number of alphabets 
            # To the power of the position of the charcode 
            r += (ord(str(string)[i]) - ord('A') + 1) * 26 ** (length - i - 1)
        return r

# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--    
    def mungeskewed(train,  numeric_feats):
        '''This function checks for skewness in the categorical features and applies box-cox transformation'''
        ntrain = train.shape[0]

        # Calculating the skewness on the entire data's features 
        skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
        # seperating the features which have higher than 0.25 skewness 
        skewed_feats = skewed_feats[skewed_feats > 0.25]
        skewed_feats = skewed_feats.index

        # Transforming all the highly skewed variables with BOX-cox 
        # Data leakage is avoided by checking the skewness on train_data only and skipping the test data
        for feats in skewed_feats:
            train[feats] = train[feats] + 1
            train[feats], lam = boxcox(train[feats])
        return train, ntrain
# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
    numeric_feats = [x for x in train_data.columns[1:-1] if 'cont' in x]
    categorical_feats = [x for x in train_data.columns[1:-1] if 'cat' in x]
    train_test, ntrain = mungeskewed(train_data,numeric_feats)
# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--    
                
    train = train_test.iloc[:ntrain, :].copy()
# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
    
    mimx_scale_data = load(open('min_max_scale.pkl', 'rb'))
    
    def min_max_scaler(data,var):
    #     print("initiated")
        scaled_data=[]
        for i in ( range (0, len(data))):
            X_std = (float(data[var].iloc[i]) - float(mimx_scale_data[var].min())) / (float(mimx_scale_data[var].max()) - float(mimx_scale_data[var].min()))

            scaled_data.append(float(X_std))
        return scaled_data
        
    # Referenced from Ali's script (https://www.kaggle.com/aliajouz/allstate-claims-severity/singel-model-lb-1117)


    train["cont1"] = np.sqrt(min_max_scaler(train ,"cont1"))
    train["cont4"] = np.sqrt(min_max_scaler(train ,"cont4"))
    train["cont5"] = np.sqrt(min_max_scaler(train ,"cont5"))
    train["cont8"] = np.sqrt(min_max_scaler(train ,"cont8"))
    train["cont10"] = np.sqrt(min_max_scaler(train ,"cont10"))
    train["cont11"] = np.sqrt(min_max_scaler(train ,"cont11"))
    train["cont12"] = np.sqrt(min_max_scaler(train ,"cont12"))
    
    train["cont6"] = np.log(min_max_scaler(train ,"cont6") )
    train["cont7"] = np.log(min_max_scaler(train ,"cont7") )
    train["cont9"] = np.log(min_max_scaler(train ,"cont9") )
    train["cont13"] = np.log(min_max_scaler(train ,"cont13"))
    train["cont14"] = (np.maximum(train["cont14"] - 0.179722, 0) / 0.665122) ** 0.25

# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
    
#     print('Combining Columns')

    for comb in ( itertools.combinations(top_cat_feats, 2)):
        feat = comb[0] + "_" + comb[1]
        train[feat] = train[comb[0]] + train[comb[1]]
        train[feat] = train[feat].apply(encode)

#     print('Encoding columns')
    for col in (categorical_feats):
        train[col] = train[col].apply(encode)

    ss = load(open('tot_data_scale.pkl', 'rb'))
    train[numeric_feats] = ss.fit_transform(train[numeric_feats].values)
    
# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
    auto_res = load(open('auto_scaler.pkl', 'rb'))
    train1=train_data.drop(['id'],axis=1).copy()
    k=[]
#     enc_dict={}
#     list(auto_res.values)
    for col in train1.select_dtypes(include=['object']).columns:
        enc = auto_res[str(col)]
        train1[col] =enc.transform(train1[col])
        
    encoder = load_model('encoder1.h5',compile=False)

    encoder.run_eagerly = True
    X_train_encode=encoder.predict(train1)

    train_final = np.concatenate((train.drop(['id'],axis=1), X_train_encode), axis=1)
    
# -*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
    d_train = xgb.DMatrix(train_final)
    modell = load(open('xgb_model.pkl', 'rb'))
    
    predictions=modell.predict(d_train)
    shift=200
    print(f'Time: {time.time() - start}')
    return  np.exp(predictions)-shift
    
    


In [53]:
final_fun_1(test_data.iloc[:2])

Time: 0.5991108417510986


array([2206.7627, 1957.6252], dtype=float32)

In [54]:
def final_fun_2(X,y): 
    predictions=final_fun_1(X)
    return  mean_absolute_error(predictions,y)
    

In [56]:
final_fun_2(test_data[:10],test_data['loss'][:10])

Time: 0.6161520481109619


944.9963090820313

In [59]:
file_name = "test_copy.txt"
with open(file_name, 'r') as f:
    file_content = f.read()

In [62]:
type(file_content)

str

In [63]:
data = pd.read_csv("test_copy.txt", sep=',')

In [64]:
data

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125541,587617,A,A,A,B,A,A,A,A,A,...,0.281143,0.438917,0.815941,0.39455,0.48740,0.40666,0.550529,0.538473,0.298734,0.345946
125542,587621,A,A,A,A,B,B,A,B,A,...,0.674529,0.346948,0.424968,0.47669,0.25753,0.26894,0.324486,0.352251,0.490001,0.290576
125543,587627,B,B,A,A,B,A,A,A,B,...,0.794794,0.808958,0.511502,0.72299,0.94438,0.83510,0.933174,0.926619,0.848129,0.808125
125544,587629,A,A,A,A,A,B,A,B,A,...,0.302678,0.372125,0.388545,0.31796,0.32128,0.36974,0.307628,0.301921,0.608259,0.361542


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('train.csv')

In [4]:

data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
